In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

C:\Users\mook\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
x_data = [[2., 5., 6., 8.],
          [5., 7., 9., 32.],
          [1., 2., 4., 81.],
          [3., 4., 17., 26.],
          [1., 3., 14., 29.], 
          [10., 19., 31.,38.]]

y_data = [[0, 1, 1],
          [1, 0, 1],
          [0, 0, 0],
          [1, 1, 1],
          [1, 0, 0],
          [1, 0, 1]]


X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])

W = tf.Variable(tf.random_normal([4, 3]))
b = tf.Variable(tf.random_normal([3]))

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)   # tf.nn.softmax만 추가했다
# cost = -tf.reduce_mean(tf.reduce_sum(y * tf.log(hypothesis)
#                                     + (1-Y)*tf.log(1-hypothesis)))

cost = -tf.reduce_mean(tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.00001).minimize(cost)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(3000):
    cost_v, _ = sess.run([cost, optimizer], feed_dict={X:x_data,Y:y_data})
    if step %100 ==0 :
        print('step', step, 'cost',cost_v)

step 0 cost 15.858963
step 100 cost 15.162211
step 200 cost 14.527119
step 300 cost 13.961766
step 400 cost 13.46893
step 500 cost 13.032242
step 600 cost 12.632846
step 700 cost 12.261291
step 800 cost 11.918492
step 900 cost 11.612668
step 1000 cost 11.352933
step 1100 cost 11.141338
step 1200 cost 10.970474
step 1300 cost 10.828606
step 1400 cost 10.705262
step 1500 cost 10.593166
step 1600 cost 10.487798
step 1700 cost 10.386473
step 1800 cost 10.287615
step 1900 cost 10.1902895
step 2000 cost 10.093928
step 2100 cost 9.998193
step 2200 cost 9.90287
step 2300 cost 9.807834
step 2400 cost 9.712999
step 2500 cost 9.618318
step 2600 cost 9.523757
step 2700 cost 9.429292
step 2800 cost 9.334916
step 2900 cost 9.240624


In [16]:
result = sess.run(hypothesis, feed_dict={X:[[1,3,5,7], [3,6,7,9]]})
result

array([[1.4104825e-01, 7.6433981e-04, 8.5818744e-01],
       [6.7353256e-02, 1.4827460e-05, 9.3263191e-01]], dtype=float32)

In [19]:
print(sess.run(tf.argmax(result,1)))

[2 2]


In [20]:
# 데이터 받아서 하기

In [22]:
xy = np.loadtxt('zoo.csv', delimiter=',',dtype=np.float32)
xy

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 3.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 6.],
       [0., 1., 1., ..., 0., 0., 1.]], dtype=float32)

In [53]:
x_data = xy[:,:-1]
y_data = xy[:,[-1]]

In [50]:
x_data.shape

(101, 16)

In [76]:
y_data_hot = tf.one_hot(y_data,depth=7)
y_data_hot= tf.reshape(y_data_hot,[-1,7])
y_data_hot=sess.run(y_data_hot)
y_data_hot.shape

(101, 7)

In [83]:
x_train = x_data[:70,:]
x_test =x_data[70:,:]
y_train = y_data_hot[:70,:]
y_test = y_data_hot[70:,:]
x_train.shape

(70, 16)

In [97]:
X = tf.placeholder(tf.float32, shape=[None,16])
Y = tf.placeholder(tf.float32, shape=[None,7])


W = tf.Variable(tf.random_normal([16, 7]))
b = tf.Variable(tf.random_normal([7]))

#1안
#hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
#cost = -tf.reduce_mean(tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
#2안
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

###
prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [98]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(3000):
    cost_v, _ = sess.run([cost, optimizer], feed_dict={X:x_train,Y:y_train})
    if step %100 ==0 :
        print('step', step, 'cost',cost_v)
print(sess.run(accuracy, feed_dict={X:x_train,Y:y_train}))

step 0 cost 7.654683
step 100 cost 0.57132524
step 200 cost 0.32990718
step 300 cost 0.24515302
step 400 cost 0.19624351
step 500 cost 0.16216795
step 600 cost 0.13600802
step 700 cost 0.11484469
step 800 cost 0.09732686
step 900 cost 0.082817644
step 1000 cost 0.07095522
step 1100 cost 0.061395045
step 1200 cost 0.0537395
step 1300 cost 0.04758917
step 1400 cost 0.042600397
step 1500 cost 0.03850342
step 1600 cost 0.035094965
step 1700 cost 0.032223906
step 1800 cost 0.02977769
step 1900 cost 0.027671605
step 2000 cost 0.025841186
step 2100 cost 0.024236856
step 2200 cost 0.022819852
step 2300 cost 0.021559697
step 2400 cost 0.02043206
step 2500 cost 0.01941723
step 2600 cost 0.018499251
step 2700 cost 0.017664976
step 2800 cost 0.016903479
step 2900 cost 0.016205689
1.0


In [99]:
result = sess.run(accuracy, feed_dict={X:x_test, Y:y_test})
result

0.8064516

In [49]:
print(sess.run(tf.argmax(result,axis=1)))

[1]
